# ETL de correos de México

In [8]:
%pip install pymongo requests beautifulsoup4 pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import re
import zipfile
import io
import pandas as pd
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from config import MONGO_URI, DB_NAME, CATALOG_COLLECTION

# --- 1. CONFIGURACIÓN ---
URL_PAGINA = "https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.correosdemexico.gob.mx',
    'Referer': 'https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx'
}

# Las 3 columnas que nos interesan
COLUMNAS_REQUERIDAS = ['d_codigo', 'd_asenta', 'D_mnpio']

def actualizar_catalogo_cp_automatico():
    """
    Descarga el catálogo de CP de Correos, lo procesa y lo guarda
    en la colección 'catalogo_cp' de MongoDB.
    """
    with requests.Session() as s:
        
        # --- PASO 1: GET (Obtener tokens y cookies) ---
        print(f"Paso 1: Obteniendo tokens de estado desde {URL_PAGINA}...")
        try:
            response_get = s.get(URL_PAGINA, headers=HEADERS, verify=True)
            response_get.raise_for_status() 
            print("  > Tokens y cookies de sesión obtenidos.")
            
        except requests.exceptions.RequestException as e:
            print(f"Error al cargar la página inicial: {e}")
            return

        # --- PASO 2: PARSE (Extraer tokens ocultos) ---
        print("Paso 2: Extrayendo __VIEWSTATE y __EVENTVALIDATION...")
        soup = BeautifulSoup(response_get.text, 'html.parser')
        
        try:
            viewstate = soup.find('input', {'id': '__VIEWSTATE'}).get('value')
            eventvalidation = soup.find('input', {'id': '__EVENTVALIDATION'}).get('value')
            viewstategenerator = "BE1A6D2E" # De tu captura de Burp
            
            if not viewstate or not eventvalidation:
                raise ValueError("No se pudieron encontrar los tokens de estado.")
                
            print("  > ¡Tokens extraídos con éxito!")
            
        except Exception as e:
            print(f"Error al parsear el HTML: {e}")
            return

        # --- PASO 3: POST (Replicar tu captura de Burp) ---
        payload = {
            '__EVENTTARGET': '', '__EVENTARGUMENT': '', '__LASTFOCUS': '',
            '__VIEWSTATE': viewstate,
            '__VIEWSTATEGENERATOR': viewstategenerator,
            '__EVENTVALIDATION': eventvalidation,
            'cboEdo': '00', 'rblTipo': 'xls', # '00' = Todos los estados
            'btnDescarga.x': '44', 'btnDescarga.y': '16'
        }
        
        print("Paso 3: Enviando petición POST para descargar el archivo...")
        
        try:
            response_post = s.post(URL_PAGINA, headers=HEADERS, data=payload, stream=True)
            response_post.raise_for_status()
            
            if 'application/octet-stream' not in response_post.headers.get('Content-Type', ''):
                print("Error: La respuesta del servidor no fue el archivo ZIP esperado.")
                return

            print("  > ¡Respuesta ZIP recibida! Procesando en memoria...")

            # --- PASO 4: PROCESAR EL ZIP Y CARGAR A MONGODB ---
            with zipfile.ZipFile(io.BytesIO(response_post.content)) as zf:
                file_name = zf.namelist()[0]
                print(f"  > Archivo encontrado en ZIP: {file_name}")
                
                with zf.open(file_name) as f:
                    print(f"  > Leyendo archivo {file_name} usando el motor de Excel...")
                    
                    # --- ¡CORRECCIÓN FINAL! ---
                    # Leemos el archivo .xls como lo que es: un Excel.
                    # Le decimos a pandas que lea TODAS las hojas (sheet_name=None)
                    # y que la primera fila (header=0) es el encabezado.
                    # 'xlrd' es el motor para leer .xls antiguos.
                    sheets_dict = pd.read_excel(
                        f, 
                        sheet_name=None, # Lee todas las hojas
                        header=0,        # La primera fila es el encabezado
                        dtype=str,       # Lee todo como texto
                        engine='xlrd'    # Motor para .xls
                    )
                    
                    print(f"  > Se leyeron {len(sheets_dict)} hojas (estados) del archivo.")

            # --- PASO 5: CONSOLIDAR Y LIMPIAR ---
            print("Paso 5: Consolidando y limpiando datos...")
            lista_de_dataframes = []
            for nombre_hoja, df in sheets_dict.items():
                # Verificamos si las columnas requeridas existen
                if all(col in df.columns for col in COLUMNAS_REQUERIDAS):
                    df_filtrado = df[COLUMNAS_REQUERIDAS]
                    lista_de_dataframes.append(df_filtrado)
                else:
                    print(f"ADVERTENCIA: La hoja '{nombre_hoja}' no tiene las columnas esperadas. Omitiendo.")
            
            # Combinamos los DataFrames de todos los estados en uno solo
            catalogo_completo_df = pd.concat(lista_de_dataframes, ignore_index=True)
            # Eliminamos filas donde los datos clave sean nulos
            catalogo_completo_df.dropna(subset=COLUMNAS_REQUERIDAS, inplace=True)
            print(f"  > Se consolidaron {len(catalogo_completo_df)} registros válidos en total.")

            # --- PASO 6: CONECTAR Y GUARDAR EN MONGODB ---
            print("Paso 6: Conectando a MongoDB para actualizar el catálogo...")
            client = MongoClient(MONGO_URI)
            db = client[DB_NAME]
            catalogo_collection = db[CATALOG_COLLECTION] 
            
            registros = catalogo_completo_df.to_dict('records')
            
            if registros:
                print(f"  > Borrando catálogo anterior en '{CATALOG_COLLECTION}'...")
                catalogo_collection.delete_many({}) 
                
                print(f"  > Insertando {len(registros)} nuevos registros...")
                catalogo_collection.insert_many(registros)
                
                print("  > Creando índice en 'd_codigo' para búsquedas rápidas...")
                catalogo_collection.create_index("d_codigo")
                print(f"¡Catálogo de Códigos Postales '{CATALOG_COLLECTION}' actualizado en MongoDB!")
            else:
                print("No se encontraron registros válidos para cargar.")
                
            client.close()
            print("Conexión a MongoDB cerrada.")

        except requests.exceptions.RequestException as e:
            print(f"Error durante la petición POST: {e}")
        except Exception as e:
            print(f"Error procesando el archivo: {e}")

# --- Ejecutar el script ---
if __name__ == "__main__":
    actualizar_catalogo_cp_automatico()

Paso 1: Obteniendo tokens de estado desde https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx...
  > Tokens y cookies de sesión obtenidos.
Paso 2: Extrayendo __VIEWSTATE y __EVENTVALIDATION...
  > ¡Tokens extraídos con éxito!
Paso 3: Enviando petición POST para descargar el archivo...
  > ¡Respuesta ZIP recibida! Procesando en memoria...
  > Archivo encontrado en ZIP: CPdescarga.xls
  > Leyendo archivo CPdescarga.xls usando el motor de Excel...
  > Se leyeron 33 hojas (estados) del archivo.
Paso 5: Consolidando y limpiando datos...
ADVERTENCIA: La hoja 'Nota' no tiene las columnas esperadas. Omitiendo.
  > Se consolidaron 157225 registros válidos en total.
Paso 6: Conectando a MongoDB para actualizar el catálogo...
  > Borrando catálogo anterior en 'catalogo_cp'...
  > Insertando 157225 nuevos registros...
  > Creando índice en 'd_codigo' para búsquedas rápidas...
¡Catálogo de Códigos Postales 'catalogo_cp' actualizado en MongoDB!
Conexión a MongoDB ce